In [1]:
%config IPComplete.greedy=True

In [2]:
import tensorflow as tf

创建会话环境

In [3]:
sess=tf.Session()

# 数据集结构
- 一个__数据集__包含多个__元素__，每个__元素的结构__都相同。一个元素包含一个或多个`tf.Tensor`对象，这些对象被称为__组件__
- __嵌套结构__映射到__元素的结构__,元素可以是__单个张量__，__张量元组__，__张量的嵌套元组__

## 创建数据集

In [4]:
# 创建张量
tensor_4=tf.random_uniform([4])
tensor_4x10 = tf.random_uniform([4,10])
tensor_4x100=tf.random_uniform([4,100],maxval=100,dtype=tf.int32)
print("tensor_4x10:",tensor_4x10)

tensor_4x10: Tensor("random_uniform_1:0", shape=(4, 10), dtype=float32)


In [5]:
# 单个张量的数据集
dataset1=tf.data.Dataset.from_tensor_slices(tensor_4x10)
dataset1

<TensorSliceDataset shapes: (10,), types: tf.float32>

In [6]:
# 张量元组的数据集
dataset2=tf.data.Dataset.from_tensor_slices((tensor_4,tensor_4x100))
dataset2

<TensorSliceDataset shapes: ((), (100,)), types: (tf.float32, tf.int32)>

In [7]:
# 嵌套张量的数据集
dataset3=tf.data.Dataset.zip((dataset1,dataset2))
dataset3

<ZipDataset shapes: ((10,), ((), (100,))), types: (tf.float32, (tf.float32, tf.int32))>

In [8]:
# 为元素的每个组件命名通常会带来便利性
dataset_name=tf.data.Dataset.from_tensor_slices({'a':tensor_4,'b':tensor_4x100})
dataset_name

<TensorSliceDataset shapes: {a: (), b: (100,)}, types: {a: tf.float32, b: tf.int32}>

## 数据集转换
`Dataset` 转换支持任何结构的数据集

在使用 Dataset.map()、Dataset.flat_map() 和 Dataset.filter() 转换时（这些转换会对每个元素应用一个函数），元素结构决定了函数的参数

In [9]:
# dataset1 = dataset1.map(lambda x: ...)

# dataset2 = dataset2.flat_map(lambda x, y: ...)

# Note: Argument destructuring is not available in Python 3.
# dataset3 = dataset3.filter(lambda x, (y, z): ...)

## 创建迭代器

### 单次迭代器
- 使用`Dataset`对象的`make_one_shot_iterator`方法

In [10]:

dataset = tf.data.Dataset.range(100)
iterator=dataset.make_one_shot_iterator()
next_element=iterator.get_next()
# print(next_element)
for i in range(6):
    value = tf.Session().run(next_element)
    print("value={}".format(value),"i={}".format(i))
#     assert i==value

value=0 i=0
value=0 i=1
value=0 i=2
value=0 i=3
value=0 i=4
value=0 i=5


In [11]:
def print_test(sess,epoch,element):
    for i in range(epoch):
        value = sess.run(element)
#         print("value={}".format(value),"i={}".format(i))
        print("value=%d"%value,"i=%d"%i)

### 可初始化迭代器
- 在同一个`Dataset`对象进行初始化
- 使用`Dataset`对象的`make_initializable_iterator`方法

In [12]:
range_num = tf.placeholder(tf.int64)
dataset = tf.data.Dataset.range(range_num)
itor=dataset.make_initializable_iterator()
next_element=itor.get_next()

epoch_5=5
epoch_10=10

sess.run(itor.initializer,feed_dict={range_num:epoch_10})
print_test(sess,epoch_10,next_element)

sess.run(itor.initializer,feed_dict={range_num:epoch_5})
print_test(sess,epoch_5,next_element)


value=0 i=0
value=1 i=1
value=2 i=2
value=3 i=3
value=4 i=4
value=5 i=5
value=6 i=6
value=7 i=7
value=8 i=8
value=9 i=9
value=0 i=0
value=1 i=1
value=2 i=2
value=3 i=3
value=4 i=4


### 可重新初始化迭代器
- 用`tf.data.Iterator.from_structure`方法创建迭代器
- 通过多个不同的 `Dataset` 对象进行初始化
- 这些对象具有相同的结构

In [13]:
def run_iterator(init_op,element,epoch):
#     始化迭代器
    sess.run(init_op)
    for i in range(epoch):
        sess.run(element)

In [ ]:
# 首先创建多个Dataset对象
t_dataset=tf.data.Dataset.range(epoch_10)
v_dadtaset=tf.data.Dataset.range(epoch_5)

# 创建迭代器
iterator = tf.data.Iterator.from_structure(t_dataset.output_types,t_dataset.output_shapes)

# 获取迭代元素
next_element=iterator.get_next()

# 根据不同的Dataset对象，创建迭代器的初始化操作
t_d_init_op=iterator.make_initializer(t_dataset)
v_d_init_op=iterator.make_initializer(v_dadtaset)

for _ in range(epoch_5):
    run_iterator(t_d_init_op,next_element,epoch_10)
    run_iterator(v_d_init_op,next_element,epoch_5)



### 可馈送迭代器
- 用`tf.dadta.Iterator.from_string_handle`方法创建迭代器
- 可以和`tf.placeholder`一起使用，通过`feed_dict`机制，来选择所使用的`Iterator`
- 功能与__可重新初始化迭代器__相同
- 在迭代器之间切换，不需要从数据集的开头，初始化迭代器

In [ ]:
# 首先创建多个Dataset对象
t_dataset=tf.data.Dataset.range(epoch_10).map(
    lambda x: x + tf.random_uniform([], -10, 10, tf.int64)).repeat()
v_dadtaset=tf.data.Dataset.range(epoch_5)

# 创建handle
handle=tf.placeholder(tf.string,shape=[])

# 创建可馈送迭代器
iterator=tf.data.Iterator.from_string_handle(handle,t_dataset.output_types,t_dataset.output_shapes)
# 获取迭代元素
next_element = iterator.get_next()

# 可提供迭代器与各种不同的迭代器一同使用
t_iterator=t_dataset.make_one_shot_iterator()
v_iterator=v_dadtaset.make_initializable_iterator()

# 返回可评估的tensor
t_handle=sess.run(t_iterator.string_handle())
v_handle=sess.run(v_iterator.string_handle())


while True:
    for _ in range(epoch_10):
        sess.run(next_element,feed_dict={handle:t_handle})
    
    sess.run(v_iterator.initializer)
    for _ in range(epoch_5):
        sess.run(next_element,feed_dict={handle:v_handle})

### 消耗迭代器中的值
调用 `Iterator.get_next()` 并不会立即使迭代器进入下个状态。必须在 TensorFlow 表达式中使用此函数返回的 `tf.Tensor` 对象

In [ ]:
dataset = tf.data.Dataset.range(5)
iterator = dataset.make_initializer_iterator()
next_element = iterator.get_next()

result = tf.add(next_element,next_element)

sess.run(iterator.initializer)

while True:
    try:
       print(sess.run(result))
    except tf.errors.OutOfRangeError:
        print("End of dataset") 
        break

如果数据集的每个元素都具有相同的嵌套结构.

`next1`、`next2` 和 `next3` 是由同一个操作/节点（通过 `Iterator.get_next()` 创建）生成的张量

因此，评估其中任何一个张量都全使所有组件进入下个状态

迭代器消耗会在一个表达式中包含所有组件

In [ ]:
iterator = dataset3.make_initializable_iterator()

sess.run(iterator)

### 保存迭代器状态
- `tf.contrib.data.make_saveable_from_iterator`函数通过迭代器创建一个`SaveableObject`，该对象可用于保存和恢复迭代器的当前状态
- __可保存对象__可以添加到 tf.train.Saver 变量列表或`tf.GraphKeys.SAVEABLE_OBJECTS`集合中
- 用与`tf.Variable` 相同的方式进行保存和恢复

In [ ]:
# 创建一个可保存迭代器的对象
saveable = tf.contrib.data.make_saveable_from_iterator(iterator)

# 通过将迭代器状态添加到可保存对象集合来保存它.
tf.add_to_collection(tf.GraphKeys.SAVEABLE_OBJECTS, saveable)
saver = tf.train.Saver()

with tf.Session() as sess:

  if should_checkpoint:
    saver.save(path_to_checkpoint)

# 恢复迭代器状态
with tf.Session() as sess:
  saver.restore(sess, path_to_checkpoint)

## 读取输入数据

### 消耗NumPy数组
